# How to Fine-tuning Gemma for Spoken Language Tasks

This notebook demonstrate how to fine tune Gemma for the specific task on replying to email requests that a Korean bakery business might get.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/kjacone/swahili_cake_boss/blob/main/Gemma/spoken-language-tasks/k-gemma-it/spoken_language_tasks_with_gemma.ipynb#scrollTo=YNDq8NbCY7oh"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **L4** or **A100 GPU**.


### Gemma setup on Kaggle
To complete this tutorial, you'll first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

* Get access to Gemma on kaggle.com.
* Select a Colab runtime with sufficient resources to run the Gemma 2B model.
* Generate and configure a Kaggle username and API key.

After you've completed the Gemma setup, move on to the next section, where you'll set environment variables for your Colab environment.

### Set environemnt variables

Set environement variables for ```KAGGLE_USERNAME``` and ```KAGGLE_KEY```.

In [7]:
import os
from google.colab import userdata, drive

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

# Mounting gDrive for to store artifacts
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Install dependencies

Install Keras and KerasNLP

In [ ]:
!pip install -q -U keras-nlp datasets
!pip install -q -U keras

# Set the backbend before importing Keras
os.environ["KERAS_BACKEND"] = "jax"
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

import keras_nlp
import keras

# Run at half precision.
#keras.config.set_floatx("bfloat16")

# Training Configurations
token_limit = 512
num_data_limit = 100
lora_name = "bankingassistant"
lora_rank = 4
lr_value = 1e-4
train_epoch = 20
model_id = "gemma2_instruct_2b_en"

##Load And Translate Data

In [ ]:
!pip install transformers datasets


In [ ]:
!pip install huggingface-hub

In [ ]:
!huggingface-cli login

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import json
import os

# Step 1: Download and save the Korean Cake Boss dataset locally
# Load the dataset from Hugging Face and save it as JSON lines for offline access
ds = load_dataset("bebechien/korean_cake_boss")
os.makedirs("korean_cake_boss_local", exist_ok=True)

# Save each split as a JSON lines file
for split in ds:
    with open(f"korean_cake_boss_local/{split}.jsonl", "w", encoding="utf-8") as f:
        for item in ds[split]:
            json.dump(item, f, ensure_ascii=False)
            f.write("\n")  # Newline to separate JSON objects

print("Dataset downloaded and saved locally.")



In [ ]:

# Step 2: Load the locally saved dataset
def load_local_dataset(filename):
    with open(filename, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f]
    return Dataset.from_dict({"input": [entry["input"] for entry in data], "output": [entry["output"] for entry in data]})

train_ds = load_local_dataset("korean_cake_boss_local/train.jsonl")
test_ds = load_local_dataset("korean_cake_boss_local/test.jsonl")
local_dataset = DatasetDict({"train": train_ds, "test": test_ds})

# Step 3: Set up the translation models
# Load Korean-to-English and English-to-Swahili translation models
model_ko_en = "Helsinki-NLP/opus-mt-ko-en"
tokenizer_ko_en = AutoTokenizer.from_pretrained(model_ko_en)
model_ko_en = AutoModelForSeq2SeqLM.from_pretrained(model_ko_en)

model_en_sw = "Helsinki-NLP/opus-mt-en-sw"
tokenizer_en_sw = AutoTokenizer.from_pretrained(model_en_sw)
model_en_sw = AutoModelForSeq2SeqLM.from_pretrained(model_en_sw)

In [ ]:

# Translation function
def translate_korean_to_swahili(text, tokenizer_ko_en, model_ko_en, tokenizer_en_sw, model_en_sw):
    # Korean to English
    inputs = tokenizer_ko_en(text, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model_ko_en.generate(**inputs)
    english_text = tokenizer_ko_en.decode(outputs[0], skip_special_tokens=True)

    # English to Swahili
    inputs = tokenizer_en_sw(english_text, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model_en_sw.generate(**inputs)
    swahili_text = tokenizer_en_sw.decode(outputs[0], skip_special_tokens=True)

    return swahili_text

In [ ]:

# Translate and save function
def translate_and_save(dataset_split, filename, tokenizer_ko_en, model_ko_en, tokenizer_en_sw, model_en_sw):
    with open(filename, "w", encoding="utf-8") as f:
        for item in dataset_split:
            translated_input = translate_korean_to_swahili(item["input"], tokenizer_ko_en, model_ko_en, tokenizer_en_sw, model_en_sw)
            translated_output = translate_korean_to_swahili(item["output"], tokenizer_ko_en, model_ko_en, tokenizer_en_sw, model_en_sw)
            # Save each translated entry as a dictionary in JSON format
            json.dump({"input": translated_input, "output": translated_output}, f, ensure_ascii=False)
            f.write("\n")  # Newline to separate JSON objects

# Translate and save the train and test splits
translate_and_save(local_dataset["train"], "korean_cake_boss_train_sw.json", tokenizer_ko_en, model_ko_en, tokenizer_en_sw, model_en_sw)
translate_and_save(local_dataset["test"], "korean_cake_boss_test_sw.json", tokenizer_ko_en, model_ko_en, tokenizer_en_sw, model_en_sw)

print("Translation completed and saved to korean_cake_boss_train_sw.json and korean_cake_boss_test_sw.json.")


In [ ]:

# Optional: Load the translated JSON files back to Dataset objects for further processing or uploading
translated_train = load_local_dataset("korean_cake_boss_train_sw.json")
translated_test = load_local_dataset("korean_cake_boss_test_sw.json")
translated_dataset = DatasetDict({"train": translated_train, "test": translated_test})

# Push the translated dataset to Hugging Face Hub
translated_dataset.push_to_hub("jacone/swahili_cake_boss", private=False)

print("Swahili-translated dataset successfully pushed to Hugging Face Hub.")

## Load Model

In [ ]:
import keras
import keras_nlp

import time

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(model_id)
gemma_lm.summary()

tick_start = 0

def tick():
    global tick_start
    tick_start = time.time()

def tock():
    print(f"TOTAL TIME ELAPSED: {time.time() - tick_start:.2f}s")

def text_gen(prompt):
    tick()
    input = f"<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"
    output = gemma_lm.generate(input, max_length=token_limit)
    print("\nGemma output:")
    print(output)
    tock()

# inference before fine-tuning
text_gen("Tafadhali andika jibu la barua pepe kwa:\n\"Hujambo, ningependa kuagiza keki moja nambari 3 kwa ajili ya maadhimisho ya harusi yetu. Je, hilo linawezekana?\n")

## Load Dataset

In [ ]:
import keras
import keras_nlp
import datasets

tokenizer = keras_nlp.models.GemmaTokenizer.from_preset(model_id)

# prompt structure
# <start_of_turn>user
# 다음에 대한 이메일 답장을 작성해줘.
# "{EMAIL CONTENT FROM THE CUSTOMER}"
# <end_of_turn>
# <start_of_turn>model
# {MODEL ANSWER}<end_of_turn>

# input, output
from datasets import load_dataset
ds = load_dataset(
    "bebechien/korean_cake_boss",
    split="train",
)
print(ds)
data = ds.with_format("np", columns=["input", "output"], output_all_columns=False)
train = []

for x in data:
  item = f"<start_of_turn>user\n다음에 대한 이메일 답장을 작성해줘.\n\"{x['input']}\"<end_of_turn>\n<start_of_turn>model\n{x['output']}<end_of_turn>"
  length = len(tokenizer(item))
  # skip data if the token length is longer than our limit
  if length < token_limit:
    train.append(item)
    if(len(train)>=num_data_limit):
      break

print(len(train))
print(train[0])
print(train[1])
print(train[2])

## LoRA Fine-tuning

In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=lora_rank)
gemma_lm.summary()

# Limit the input sequence length (to control memory usage).
gemma_lm.preprocessor.sequence_length = token_limit
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=lr_value,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)


Note that enabling LoRA reduces the number of trainable parameters significantly.

In [ ]:
class CustomCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    model_name = f"/content/drive/MyDrive/{lora_name}_{lora_rank}_epoch{epoch+1}.lora.h5"
    gemma_lm.backbone.save_lora_weights(model_name)

    # Evaluate
    text_gen("다음에 대한 이메일 답장을 작성해줘.\n\"안녕하세요, 결혼기념일을 위해 3호 케이크 1개를 주문하고 싶은데 가능할까요?\"")

history = gemma_lm.fit(train, epochs=train_epoch, batch_size=2, callbacks=[CustomCallback()])

import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.show()

In [ ]:
# Example Code for Load LoRA
'''
train_epoch=17
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(model_id)
# Use the same LoRA rank that you trained
gemma_lm.backbone.enable_lora(rank=4)

# Load pre-trained LoRA weights
gemma_lm.backbone.load_lora_weights(f"/content/drive/MyDrive/{lora_name}_{lora_rank}_epoch{train_epoch}.lora.h5")
'''

## Try a different sampler

The top-K algorithm randomly picks the next token from the tokens of top K probability.

In [ ]:
gemma_lm.compile(sampler="top_k")
text_gen("다음에 대한 이메일 답장을 작성해줘.\n\"안녕하세요, 결혼기념일을 위해 3호 케이크 1개를 주문하고 싶은데 가능할까요?\"")
text_gen("다음에 대한 이메일 답장을 작성해줘.\n\"안녕하세요, 결혼기념일을 위해 3호 케이크 1개를 주문하고 싶은데 가능할까요?\"")
text_gen("다음에 대한 이메일 답장을 작성해줘.\n\"안녕하세요, 결혼기념일을 위해 3호 케이크 1개를 주문하고 싶은데 가능할까요?\"")

Try a slight different prompts

In [ ]:
text_gen("다음에 대한 답장을 작성해줘.\n\"안녕하세요, 결혼기념일을 위해 3호 케이크 1개를 주문하고 싶은데 가능할까요?\"")
text_gen("아래에 적절한 답장을 써줘.\n\"안녕하세요, 결혼기념일을 위해 3호 케이크 1개를 주문하고 싶은데 가능할까요?\"")
text_gen("다음에 관한 답장을 써주세요.\n\"안녕하세요, 결혼기념일을 위해 3호 케이크 1개를 주문하고 싶은데 가능할까요?\"")

Try a different email inputs

In [ ]:
text_gen("""다음에 대한 이메일 답장을 작성해줘.
"안녕하세요,

6월 15일에 있을 행사 답례품으로 쿠키 & 머핀 세트를 대량 주문하고 싶습니다.

수량: 50세트
구성: 쿠키 2개 + 머핀 1개 (개별 포장)
디자인: 심플하고 고급스러운 디자인 (리본 포장 등)
문구: "감사합니다" 스티커 부착
배송 날짜: 6월 14일
대량 주문 할인 혜택이 있는지, 있다면 견적과 함께 배송 가능 여부를 알려주시면 감사하겠습니다.

감사합니다.

박철수 드림" """)
